In [ ]:
from ml_project.utils import files

# Correct the Order of the columns

files.save_points_df(files.load_points_df(),override=True)

files.save_samples_df(files.load_samples_df(),override=True)